In [44]:
import pandas as pd
import numpy as np
from scipy.sparse import lil_matrix
from cvxopt import matrix, solvers
solvers.options['show_progress'] = False
from tqdm import tqdm

In [45]:
class KmerKernel:
    def __init__(self, kmin=7, kmax=20):
        """
        Parameters
        ----------
        kmin : int
            Minimum k-mer length.
        kmax : int
            Maximum k-mer length.
        """
        self.kmin = kmin
        self.kmax = kmax
        self.vocab = {}

    def _generate_kmers(self, seq):
        """
        Generate k-mers from a sequence.

        Parameters
        ----------
        seq : str
            Sequence.

        Returns
        -------
        list
            List of k-m
        """
        kmers = []
        for k in range(self.kmin, self.kmax + 1):
            kmers.extend(seq[i:i+k] for i in range(len(seq) - k + 1))
        return kmers
    
    def fit(self, sequences):
        kmer_set = set()
        for seq in sequences:
            kmer_set.update(self._generate_kmers(seq))
        self.vocab = {kmer: idx for idx, kmer in enumerate(sorted(kmer_set))}
        self.phi = self.get_phi(sequences)
    
    def get_phi(self, sequences):
        phi = lil_matrix((len(sequences), len(self.vocab)), dtype=int)
        for i, seq in enumerate(sequences):
            kmers = self._generate_kmers(seq)
            for kmer in kmers:
                if kmer in self.vocab:
                    phi[i, self.vocab[kmer]] += 1
        return phi.tocsr()
    
    def get_kernel(self):
        return (self.phi @ self.phi.T / np.sqrt(self.phi.sum(axis=1) @ self.phi.sum(axis=1).T)).todense()
    
    def predict(self, sequences):
        phis = self.get_phi(sequences)
        k_train = self.get_kernel()
        div = np.sqrt(k_train).sum(axis=0)
        
        k_test = (phis @ phis.T).todense()
        pred = []
        for i in range(phis.shape[0]):
            phi_x = phis[i, :]
            k_xx = k_test[i, i]
            pred.append((phi_x @ self.phi.T / (div*np.sqrt(k_xx))).todense().flatten())
        return np.array(pred).squeeze()
        # return (phi_x @ self.phi.T / (np.sqrt(self.phi @ self.phi.T) * np.sqrt(phi_x @ phi_x.T)).sum(axis=1)).todense()
        

In [46]:
class Dataset:
    def __init__(self, train_idx = 0, mode = 'train'):
        if mode == 'train':
            self.df = pd.read_csv(f'data/Xtr{train_idx}.csv', index_col=0)
            self.target = pd.read_csv(f'data/Ytr{train_idx}.csv', index_col=0)
        else:
            self.df = pd.read_csv(f'data/Xte{train_idx}.csv', index_col=0)
            self.target = None

    def train_test_split(self, test_size=0.2, shuffle=True):
        # without using sklearn
        n = len(self.df)
        indices = np.arange(n)
        if shuffle:
            np.random.shuffle(indices)
        n_test = int(n * test_size)
        test_indices = indices[:n_test]
        train_indices = indices[n_test:]
        train_df = self.df.iloc[train_indices].seq.to_numpy()
        test_df = self.df.iloc[test_indices].seq.to_numpy()
        if self.target is not None:
            train_target = 2 * self.target.iloc[train_indices].Bound.to_numpy() - 1
            test_target = 2 * self.target.iloc[test_indices].Bound.to_numpy() - 1
        else:
            train_target = None
            test_target = None
        return train_df, test_df, train_target, test_target

In [47]:
class Score:
    """Score object to evaluate a method

    """
    def __init__(self, pred, labels):
        self.n = len(labels)
        self.tp = np.sum(np.logical_and(pred == 1., labels == 1.))
        self.fn = np.sum(np.logical_and(pred == -1., labels == 1.))
        self.fp = np.sum(np.logical_and(pred == 1., labels == -1.))
        self.recall = self.tp / (self.fn + self.tp)
        self.precision = self.tp / (self.fp + self.tp)
        self.accuracy = np.sum(labels == pred) / self.n

    def __str__(self):
        acc = "Accuracy: " + str(self.accuracy)
        pre = "Precision: " + str(self.precision)
        rec = "Recall: " + str(self.recall)
        return ", ".join([acc, pre, rec])
    
    def __repr__(self):
        return self.__str__()

class KSVM():
    def __init__(self,
                 kernel_cls,
                 C,
                 tol,
                 name="KSVM"):
        
        self.kernel = kernel_cls()
        self.__name__ = name

        self.C = C
        self.tol = tol

    def predict(self, data):
        K_xi = self.kernel.predict(data)
        pred = self.alpha @ K_xi.T + self.b
        return np.sign(pred)

    def score_recall_precision(self, data, labels):

        mask = np.arange(len(data))
        np.random.shuffle(mask)
        pred = self.predict(data[mask])
        score = Score(pred, labels[mask])
        return score
    
    def fit(self, data, labels):
        self.kernel.fit(data)
        K = self.kernel.get_kernel()
        n = len(K)
        y = labels
        C = self.C

        G_top = np.diag(np.ones(n) * (-1))
        h_left = np.zeros(n)
        G_bot = np.eye(n)
        h_right = np.ones(n) * C
        G = matrix(np.vstack([G_top, G_bot]), (2 * n, n), 'd')
        h = matrix(np.hstack([h_left, h_right]), (2 * n, 1), 'd')
        P = matrix(np.dot(np.diag(y), np.dot(K, np.diag(y))), (n, n), 'd')
        q = matrix(np.ones(n) * (-1), (n, 1), 'd')

        A = matrix(y, (1, n), "d")

        b = matrix(0.0)
        alpha = y * np.array(solvers.qp(P, q, G, h, A=A, b=b)["x"]).reshape(-1)

        support_vectors = np.where(np.abs(alpha) > self.tol)[0]

        intercept = 0
        for sv in support_vectors:
            intercept += y[sv]
            intercept -= np.sum(
                alpha[support_vectors, None] * K[sv, support_vectors])
        if len(support_vectors) > 0:
            intercept /= len(support_vectors)

        # set to zero non support vectors
        alpha[np.where(np.abs(alpha) <= self.tol)[0]] = 0
        
        self.b = intercept
        self.alpha = alpha
        return self.alpha, self.b

In [48]:
from functools import partial

dataset = Dataset(train_idx=0)
Xtr, Xte, Ytr, Yte = dataset.train_test_split(test_size=0.2)
ksvm = KSVM(partial(KmerKernel, kmin=7, kmax=20), C=0.1, tol=1e-5)
alpha, beta = ksvm.fit(Xtr, Ytr)

KeyboardInterrupt: 

In [38]:
ksvm.score_recall_precision(Xtr, Ytr)

100%|██████████| 1600/1600 [00:24<00:00, 65.11it/s]


Accuracy: 0.944375, Precision: 1.0, Recall: 0.8877679697351829

In [39]:
ksvm.score_recall_precision(Xte, Yte)

100%|██████████| 400/400 [00:06<00:00, 63.67it/s]


Accuracy: 0.6075, Precision: 0.574585635359116, Recall: 0.5652173913043478

In [50]:
import optuna
from tqdm import tqdm

cv = 3  
best_params_per_idx = {}

for train_idx in range(3):
    print(f"Optimizing hyperparameters for train_idx={train_idx}")
    dataset = Dataset(train_idx=train_idx)

    def objective(trial):
        kmin = trial.suggest_categorical("kmin", [3, 5, 7, 9, 11])
        kmax = trial.suggest_categorical("kmax", [12, 15, 18, 20, 22, 25])
        C = trial.suggest_categorical("C", [0.01, 0.1, 1, 2, 5])

        if kmin >= kmax:
            return 0  # Invalid combination

        mean_accuracy = 0
        for _ in range(cv):
            Xtr, Xte, Ytr, Yte = dataset.train_test_split(test_size=0.2)
            ksvm = KSVM(partial(KmerKernel, kmin=kmin, kmax=kmax), C=C, tol=1e-5)
            alpha, beta = ksvm.fit(Xtr, Ytr)
            score = ksvm.score_recall_precision(Xte, Yte)
            mean_accuracy += score.accuracy
        
        return mean_accuracy / cv

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=50, show_progress_bar=True, n_jobs=1)

    best_params_per_idx[train_idx] = {"SVM": study.best_params}
    print(f"Best parameters for train_idx={train_idx}: {study.best_params}, Accuracy={study.best_value:.4f}")

# Summary of best parameters
print("Final best parameters:")
for idx, params in best_params_per_idx.items():
    print(f"train_idx={idx}: SVM: {params['SVM']}")


[I 2025-02-27 16:31:29,496] A new study created in memory with name: no-name-81a6a60e-6f26-439c-85ae-5ec902dac694


Optimizing hyperparameters for train_idx=0


  0%|          | 0/50 [00:00<?, ?it/s]

/var/folders/ck/j7s268jx3znc22kgclrw1rcc0000gn/T/ipykernel_25602/2302153764.py:11: RuntimeWarning: invalid value encountered in scalar divide
  self.precision = self.tp / (self.fp + self.tp)


[I 2025-02-27 16:32:50,837] Trial 0 finished with value: 0.5108333333333334 and parameters: {'kmin': 5, 'kmax': 20, 'C': 0.01}. Best is trial 0 with value: 0.5108333333333334.
[I 2025-02-27 16:33:39,383] Trial 1 finished with value: 0.545 and parameters: {'kmin': 9, 'kmax': 12, 'C': 0.01}. Best is trial 1 with value: 0.545.


...

  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-02-27 18:40:03,444] Trial 0 finished with value: 0.6333333333333333 and parameters: {'kmin': 7, 'kmax': 22, 'C': 0.1}. Best is trial 0 with value: 0.6333333333333333.
[I 2025-02-27 18:40:55,973] Trial 1 finished with value: 0.6124999999999999 and parameters: {'kmin': 5, 'kmax': 12, 'C': 5}. Best is trial 0 with value: 0.6333333333333333.
[I 2025-02-27 18:42:20,202] Trial 2 finished with value: 0.6333333333333333 and parameters: {'kmin': 7, 'kmax': 25, 'C': 2}. Best is trial 0 with value: 0.6333333333333333.
[I 2025-02-27 18:43:26,722] Trial 3 finished with value: 0.6133333333333333 and parameters: {'kmin': 7, 'kmax': 18, 'C': 0.01}. Best is trial 0 with value: 0.6333333333333333.


/Users/noe/opt/anaconda3/envs/mlp/lib/python3.11/site-packages/scipy/sparse/_base.py:704: RuntimeWarning: divide by zero encountered in divide
  recip = np.true_divide(1., other)


[I 2025-02-27 18:44:38,305] Trial 4 finished with value: 0.645 and parameters: {'kmin': 11, 'kmax': 25, 'C': 1}. Best is trial 4 with value: 0.645.
[I 2025-02-27 18:46:08,489] Trial 5 finished with value: 0.6233333333333334 and parameters: {'kmin': 5, 'kmax': 25, 'C': 0.1}. Best is trial 4 with value: 0.645.
[I 2025-02-27 18:47:18,107] Trial 6 finished with value: 0.6775000000000001 and parameters: {'kmin': 5, 'kmax': 20, 'C': 2}. Best is trial 6 with value: 0.6775000000000001.
[I 2025-02-27 18:48:32,629] Trial 7 finished with value: 0.5766666666666667 and parameters: {'kmin': 3, 'kmax': 20, 'C': 1}. Best is trial 6 with value: 0.6775000000000001.
[I 2025-02-27 18:49:50,088] Trial 8 finished with value: 0.5858333333333333 and parameters: {'kmin': 3, 'kmax': 18, 'C': 0.01}. Best is trial 6 with value: 0.6775000000000001.
[I 2025-02-27 18:50:56,935] Trial 9 finished with value: 0.6574999999999999 and parameters: {'kmin': 5, 'kmax': 18, 'C': 2}. Best is trial 6 with value: 0.6775000000000

KeyboardInterrupt: 